In [34]:
import sys
import os

local_acoular_path = r'D:\Python\Projekt_acoular\acoular'
if local_acoular_path not in sys.path:
    sys.path.insert(0, local_acoular_path)

# Überprüfen, ob der Pfad korrekt ist
#print(f"Pfad hinzugefügt: {local_acoular_path}")
#print(f"sys.path: {sys.path}")

In [28]:
#from acoular import LoudnessStationary, LoudnessTimevariant

DEMO -- Loudness in acoular

Beispiel 1: Beamforming

In [35]:
#import and file path
import acoular
from numpy import zeros
from pylab import colorbar, figure, imshow, show, subplot, tight_layout, title

micgeofile = r'D:\Python\Projekt_acoular\acoular\acoular\xml\array_56.xml'
datafile = r'D:\Python\Projekt_acoular\acoular\examples\example_loudness\example_data.h5'
calibfile = r'D:\Python\Projekt_acoular\acoular\examples\example_loudness\example_calib.xml'
cfreq = 4000

In [36]:
#Prepare Data for Beamforming: get Microfone geometry, get timesaples of measured data, set grid to analyse, set steering vector map
mg = acoular.MicGeom(from_file=micgeofile)
ts = acoular.MaskedTimeSamples(name=datafile)
ts.start = 0
ts.stop = 16000
rg = acoular.RectGrid(x_min=-0.6, x_max=0.0, y_min=-0.3, y_max=0.3, z=0.68, increment=0.1)
env = acoular.Environment(c=346.04)
st = acoular.SteeringVector(grid=rg, mics=mg, env=env)


In [37]:
#execute Beamforming in time domaine
bt = acoular.BeamformerTime(source=ts, steer=st)

#extract timesample data from beamforming -> nicht nötig, wird in loudness gemacht
data = bt.result()

In [31]:
ld = acoular.LoudnessStationary(source=bt)

In [28]:
file = r'D:\Python\Projekt_acoular\acoular\examples\example_loudness\three_sources.h5'
ts = acoular.TimeSamples(name= file)
tio = acoular.TimeInOut(source = ts)
ld = acoular.LoudnessStationary(source = tio)


In [32]:
ld.specific_loudness


array([], dtype=int32)

In [40]:
n=0
for r in ts.result(2048):
    print('Timesample', len(r))
    n=n+1
n=0
for r in bt.result():
    print('Beamform',r.shape)
    n=n+1

Timesample 2048
Timesample 2048
Timesample 2048
Timesample 2048
Timesample 2048
Timesample 2048
Timesample 2048
Timesample 1664
Beamform (2048, 49)
Beamform (2048, 49)
Beamform (2048, 49)
Beamform (2048, 49)
Beamform (2048, 49)
Beamform (2048, 49)
Beamform (2048, 49)
Beamform (1443, 49)


In [14]:
import numpy
numpy.__version__

'1.24.0'

In [ ]:
# ===============================================================================
# plot result maps for different beamformers in time domain
# ===============================================================================
i2 = 1  # no of figure
i1 = 1  # no of subplot
for b in (cacht, cachts, cachct, cachcts):
    # first, plot time-dependent result (block-wise)
    figure(i2, (7, 7))
    i2 += 1
    res = zeros(rg.size)  # init accumulator for average
    i3 = 1  # no of subplot
    for r in b.result(1):  # one single block
        subplot(4, 4, i3)
        i3 += 1
        res += r[0]  # average accum.
        map = r[0].reshape(rg.shape)
        mx = acoular.L_p(map.max())
        imshow(acoular.L_p(map.T), vmax=mx, vmin=mx - 15, origin='lower', interpolation='nearest', extent=rg.extend())
        title('%i' % ((i3 - 1) * 1024))
    res /= i3 - 1  # average
    tight_layout()

    # second, plot overall result (average over all blocks)
    figure(10)
    subplot(4, 4, i1)
    i1 += 1
    map = res.reshape(rg.shape)
    mx = acoular.L_p(map.max())
    imshow(acoular.L_p(map.T), vmax=mx, vmin=mx - 15, origin='lower', interpolation='nearest', extent=rg.extend())
    colorbar()
    title(('BeamformerTime', 'BeamformerTimeSq', 'BeamformerCleant', 'BeamformerCleantSq')[i2 - 2])
tight_layout()
# only display result on screen if this script is run directly
if __name__ == '__main__':
    show()